# Bibliotecas

In [5]:
# Import required libraries

# linear algebra and data processing libraries
import numpy as np
import pandas as pd

# scikit-learn libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Graphics Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Utils
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Preparação e Limpeza dos Dados

Nesta etapa, realizamos o processamento inicial do dataset `boardgames.csv`:

1.  **Carregamento:** Leitura dos dados brutos.
2.  **Seleção de Features:** Filtragem das colunas relevantes para a análise.
3.  **Engenharia de Atributos:** Criação da variável `rating_category` para classificar os jogos baseados na nota média (*average*).

In [6]:
# Load dataset

games_df = pd.read_csv('boardgames.csv')

#Filtrar colunas desnecessárias

cols_to_keep = [
    'primary',
    'yearpublished',
    'minplayers',
    'maxplayers',
    'minplaytime',
    'minage',
    'boardgamecategory',
    'boardgamemechanic',
    'boardgamefamily',
    'boardgamedesigner',
    'boardgameartist',
    'boardgamepublisher',
    'usersrated',
    'bayesaverage',
    'playingtime',
    'averageweight',
    'average'
]

games_df = games_df[cols_to_keep]

# Definição dos Novos Limites (Bins) e Rótulos (Labels)
bins = [0, 4, 6.2, 7.5, 10.1]
labels = ['bad', 'mediocre', 'good', 'excelent']

# Criação da nova coluna 'rating_category'
games_df['rating_category'] = pd.cut(
    games_df['average'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

print("Nr. rows - train: ", len(games_df))

Nr. rows - train:  21632


# Unsupervised Apriori

In [ ]:

use_cols = ["primary", "boardgamecategory", "boardgamemechanic"]
adf = games_df[use_cols].dropna().copy()

print("Rows após dropna:", len(adf))

def parse_list(s):
    if pd.isna(s):
        return []
    if isinstance(s, list):
        return [str(x).strip() for x in s if str(x).strip()]
    s = str(s).strip()
    try:
        lst = ast.literal_eval(s)          # caso "['A','B']"
        if isinstance(lst, list):
            return [str(x).strip() for x in lst if str(x).strip()]
    except:
        pass
    if "," in s:
        return [x.strip() for x in s.split(",") if x.strip()]
    return [s] if s else []

transactions = []
for _, row in adf.iterrows():
    cats = parse_list(row["boardgamecategory"])
    mecs = parse_list(row["boardgamemechanic"])
    items = list(dict.fromkeys(cats + mecs))  # remove duplicados
    if len(items) >= 2:
        transactions.append(items)

print("Transações criadas:", len(transactions))
print("Exemplo transação:", transactions[0][:15] if len(transactions) else "SEM TRANSAÇÕES")

te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_hot = pd.DataFrame(te_array, columns=te.columns_)

print("df_hot shape:", df_hot.shape) 

min_sup = 0.005   # 0.5%
freq = apriori(df_hot, min_support=min_sup, use_colnames=True)
freq = freq.sort_values("support", ascending=False)
print(f"Frequent itemsets (min_support={min_sup}):", len(freq))

if len(freq) == 0:
    raise ValueError(
        "Apriori não encontrou itemsets. Baixa min_support (ex: 0.002) "
        "ou verifica parsing/colunas."
    )

rules = association_rules(freq, metric="confidence", min_threshold=0.3)
rules = rules.sort_values(["lift", "confidence"], ascending=False)

rules[["antecedents","consequents","support","confidence","lift"]].head(20)

Rows após dropna: 19811
Transações criadas: 19809
Exemplo transação: ["['Medical']", "['Action Points'", "'Cooperative Game'", "'Hand Management'", "'Point to Point Movement'", "'Set Collection'", "'Trading'", "'Variable Player Powers']"]
df_hot shape: (19809, 846)
Frequent itemsets (min_support=0.005): 1143


,antecedents,consequents,support,confidence,lift
2285,"(['Auction/Bidding', 'Set Collection', 'Tradin...","('Roll / Spin and Move', 'Negotiation'])",0.005099,0.870690,141.372880
2310,"('Roll / Spin and Move', 'Negotiation'])","(['Auction/Bidding', 'Set Collection', 'Tradin...",0.005099,0.827869,141.372880
2509,"('Roll / Spin and Move', 'Negotiation'])","('Player Elimination', ['Auction/Bidding', 'Tr...",0.005048,0.819672,141.190306
2504,"('Player Elimination', ['Auction/Bidding', 'Tr...","('Roll / Spin and Move', 'Negotiation'])",0.005048,0.869565,141.190306
2289,"('Roll / Spin and Move', ['Auction/Bidding', '...","('Set Collection', 'Trading'], ['Economic')",0.005099,0.848739,138.947774
2306,"('Set Collection', 'Trading'], ['Economic')","('Roll / Spin and Move', ['Auction/Bidding', '...",0.005099,0.834711,138.947774
2498,"('Roll / Spin and Move', ['Auction/Bidding', '...","('Player Elimination', 'Trading'])",0.005048,0.840336,137.572054
2579,"('Roll / Spin and Move', ['Auction/Bidding', [...","('Player Elimination', 'Trading'])",0.005048,0.840336,137.572054
2515,"('Player Elimination', 'Trading'])","('Roll / Spin and Move', ['Auction/Bidding', '...",0.005048,0.826446,137.572054
2598,"('Player Elimination', 'Trading'])","('Roll / Spin and Move', ['Auction/Bidding', [...",0.005048,0.826446,137.572054
